In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.environ["INVEST_TOKEN"]

In [22]:
from datetime import timedelta
from pandas import DataFrame
from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

all_candles = []

def main():

    with Client(TOKEN) as client:
        for candle in client.get_all_candles(
            figi="BBG004730N88",    # FIGI Composite of SBER
            from_=now() - timedelta(days=7),
            interval=CandleInterval.CANDLE_INTERVAL_HOUR,
        ):
            all_candles.append(
                {
                    "Time": candle.time,
                    "Open": candle.open,
                    "Close": candle.close,
                    "Max": candle.high,
                    "Min": candle.low,
                    "Volume": candle.volume,
                }
            )

    return 0


if __name__ == "__main__":
    main()


In [23]:
from modules.modules import Quotation, factor
def convert_quotation(q: Quotation):
    return q.units + q.nano / factor

df = DataFrame(all_candles)
for column in ["Open", "Close", "Max", "Min"]:
    df[column] = df[column].apply(convert_quotation)

df.set_index("Time", inplace=True)

from modules.modules import Window

wi = Window(small=9, medium=26, large=52)

In [24]:
df["Tenkan-sen"]=(df["Max"].rolling(window=wi.small).max() + df["Min"].rolling(window=wi.small).min()) / 2
df['Kijun-sen'] = (df['Max'].rolling(window=wi.medium).max() + df['Min'].rolling(window=wi.medium).min()) / 2
df['Senkou Span A'] = ((df['Tenkan-sen'] + df['Kijun-sen']) / 2).shift(wi.medium)
df['Senkou Span B'] = ((df['Max'].rolling(window=wi.large).max() + df['Min'].rolling(window=wi.large).min()) / 2).shift(wi.medium)
df['Chikou Span'] = df['Close'].shift(-wi.medium)

In [25]:
df.head()

,Open,Close,Max,Min,Volume,Tenkan-sen,Kijun-sen,Senkou Span A,Senkou Span B,Chikou Span
Time,,,,,,,,,,
2024-11-17 19:00:00+00:00,249.12,249.70,250.25,247.2,87298,NaN,NaN,NaN,NaN,245.44
2024-11-17 20:00:00+00:00,248.96,248.54,249.62,248.0,45736,NaN,NaN,NaN,NaN,244.28
2024-11-18 04:00:00+00:00,248.03,248.20,248.99,246.5,15511,NaN,NaN,NaN,NaN,242.92
2024-11-18 05:00:00+00:00,248.21,248.99,249.43,248.2,14350,NaN,NaN,NaN,NaN,241.32
2024-11-18 06:00:00+00:00,249.00,249.10,249.45,248.5,59198,NaN,NaN,NaN,NaN,240.45


In [27]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['Max'],
                low=df['Min'],
                close=df['Close'],
                increasing_line_color='black',
                decreasing_line_color='grey')])

fig.add_trace(go.Scatter(x=df.index, y=df['Tenkan-sen'], mode='lines', name='Tenkan-sen', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df.index, y=df['Kijun-sen'], mode='lines', name='Kijun-sen', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span A'], mode='lines', name='Senkou Span A', line=dict(color='yellow')))
fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span B'], mode='lines', name='Senkou Span B', line=dict(color='purple')))

fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span A'], fill='tonexty', fillcolor='rgba(0,100,80,0.2)', line=dict(color='rgba(255,257,255,0)'), name='Kumo'))

fig.add_trace(go.Scatter(x=df.index, y=df['Chikou Span'], mode='lines', name='Chikou Span', line=dict(color='green')))

buySignals = (df['Tenkan-sen'] > df['Kijun-sen']) & (df['Tenkan-sen'].shift(1) <= df['Kijun-sen'].shift(1))
sellSignals = (df['Tenkan-sen'] < df['Kijun-sen']) & (df['Tenkan-sen'].shift(1) >= df['Kijun-sen'].shift(1))

fig.add_trace(go.Scatter(x=df.index[buySignals], y=df['Min'][buySignals], mode='markers', marker=dict(color='green', size=8), name='Buy Signal'))
fig.add_trace(go.Scatter(x=df.index[sellSignals], y=df['Max'][sellSignals], mode='markers', marker=dict(color='red', size=8), name='Sell Signal'))

fig.show()